##### Class Exercise - Exchange Rate

The management of AgroFarms is impressed by the efficiency of the new process of collecting weather readings. They have asked that you develop a similar process to collect exchange rates in all their currencies of interes and load this data daily to their postgres database. 

In [91]:
# Import libraries
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import datetime as dtime
from sqlalchemy import create_engine, VARCHAR, Numeric, TIMESTAMP, Float

In [94]:
load_dotenv('exch_rt.env', override=True)

True

In [108]:
# Making a test API call
api_key = os.getenv('api')
base_currency = 'NGN'
url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{base_currency}'

response = requests.get(url)

# check status
if response.status_code == 200:
    print(f"Success fetching exchange rate from {base_currency} to other currencies - Status code: {response.status_code}")
else:
    print(f"Error fetching exchange rate from {base_currency} to other currencies - Status code: {response.status_code}")

Success fetching exchange rate from NGN to other currencies - Status code: 200


In [98]:
# View the response
my_data = response.json()
display(my_data)

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1727827201,
 'time_last_update_utc': 'Wed, 02 Oct 2024 00:00:01 +0000',
 'time_next_update_unix': 1727913601,
 'time_next_update_utc': 'Thu, 03 Oct 2024 00:00:01 +0000',
 'base_code': 'NGN',
 'conversion_rates': {'NGN': 1,
  'AED': 0.002215,
  'AFN': 0.04118,
  'ALL': 0.05329,
  'AMD': 0.2326,
  'ANG': 0.001079,
  'AOA': 0.5728,
  'ARS': 0.5857,
  'AUD': 0.00086956,
  'AWG': 0.001079,
  'AZN': 0.001019,
  'BAM': 0.001053,
  'BBD': 0.001206,
  'BDT': 0.07177,
  'BGN': 0.001053,
  'BHD': 0.00022673,
  'BIF': 1.7432,
  'BMD': 0.000603,
  'BND': 0.00077278,
  'BOB': 0.004147,
  'BRL': 0.003276,
  'BSD': 0.000603,
  'BTN': 0.05034,
  'BWP': 0.007909,
  'BYN': 0.001957,
  'BZD': 0.001206,
  'CAD': 0.00081224,
  'CDF': 1.7158,
  'CHF': 0.00050718,
  'CLP': 0.5381,
  'CNY': 0.004216,
  'COP': 2.5316,
  'CRC': 0.3109,
  'CUP': 0.01

In [96]:
# Test - Extracting relevant information - assume AgroFarm's base currency is NGN and other currencies they transact with are USD, GBP, EUR, CAD, GHS, QAR
Rate_ID = my_data['time_last_update_unix']
b_curr = my_data['base_code']
USD = my_data['conversion_rates']['USD']
GBP = my_data['conversion_rates']['GBP']
EUR = my_data['conversion_rates']['EUR']
CAD = my_data['conversion_rates']['CAD']
GHS = my_data['conversion_rates']['GHS']
QAR = my_data['conversion_rates']['QAR']
Rate_Date = my_data['time_last_update_utc']
Comment = f'Rate extract result was: {my_data['result']}'

In [97]:
# Making the dataframe
curr_dict = {
    'Rate_ID' : Rate_ID,
    'Base_CURR' : b_curr,
    'USD' : USD,
    'GBP' : GBP,
    'EUR' : EUR,
    'CAD' : CAD,
    'GHS' : GHS,
    'QAR' : QAR,
    'Rate_Date' : Rate_Date,
    'Comment' : Comment
}

extract_data = pd.DataFrame([curr_dict])
extract_data.head()

,Rate_ID,Base_CURR,USD,GBP,EUR,CAD,GHS,QAR,Rate_Date,Comment
0,1727568004,NGN,0.00061,0.000453,0.000542,0.000817,0.009577,0.00222,"Sun, 29 Sep 2024 00:00:04 +0000",Rate extract result was: success


In [46]:
# Extacting exchange rates from multiple currencies - reading the currencies from a CSV file
currency_list = pd.read_csv('currencies.csv')
currency_list.head()

,AgroFarm Currencies,Code
0,1,USD
1,2,GBP
2,3,EUR
3,4,CAD
4,5,GHS


In [100]:
# Making the real API call
api_key = os.getenv('api')
base_currency = 'NGN'
rate_extract = {}

for index, row in currency_list.iterrows():
    curr = row['Code']

    url = f'https://v6.exchangerate-api.com/v6/{api_key}/pair/{base_currency}/{curr}'
    response = requests.get(url)

    if response.status_code == 200: 
        my_data = response.json()
        rate_extract[curr] = my_data['conversion_rate']
        
    else:
        print(f"Error fetching exchange rate from {base_currency} to {curr} - Status code: {response.status_code}")
        rate_extract['Comment'] = f'Rate extract result was: {my_data['result']}'
        break # if any api call fails, then the process should stop
    
rate_extract['Rate_ID'] = my_data['time_last_update_unix']
rate_extract['Rate_Date'] = my_data['time_last_update_utc']
rate_extract['Base_Currency'] = my_data['base_code']
rate_extract['Comment'] = f'Rate extract result was: {my_data['result']}'


In [101]:
rate_extracted = pd.DataFrame([rate_extract])
rate_extracted.head()

,USD,GBP,EUR,CAD,GHS,QAR,Rate_ID,Rate_Date,Base_Currency,Comment
0,0.000602,0.000449,0.000538,0.000812,0.009528,0.002195,1727827201,"Wed, 02 Oct 2024 00:00:01 +0000",NGN,Rate extract result was: success


##### My Option 1 to fix the column arrangement

Afte data is extracted into the Dataframe, update the dataframe to rearrange the columns

In [102]:
rate_extracted = pd.DataFrame([rate_extract])
rate_extracted = rate_extracted[['Rate_ID', 'Rate_Date', 'Base_Currency', 'USD', 'GBP', 'EUR', 'CAD', 'GHS', 'QAR', 'Comment']]
rate_extracted.head()

,Rate_ID,Rate_Date,Base_Currency,USD,GBP,EUR,CAD,GHS,QAR,Comment
0,1727827201,"Wed, 02 Oct 2024 00:00:01 +0000",NGN,0.000602,0.000449,0.000538,0.000812,0.009528,0.002195,Rate extract result was: success


##### My Option 2 to fix the column arrangement

From when the initial code run, arrange the columns in the desired order. This will mean that the columns that will not have data because the api is yet to be called will have a dummy data e.g. 'Nothing'.

Then within the for loop, the values will be updated. But the code may be a little bit longer

In [84]:
# Making the real API call
api_key = os.getenv('api')
base_currency = 'NGN'
rate_extract = {}
rate_extract['Rate_ID'] = 'Nothing'
rate_extract['Rate_Date'] = 'Nothing'
rate_extract['Base_Currency'] = 'Nothing'
rate_extract['Comment'] = 'Nothing'

for index, row in currency_list.iterrows():
    curr = row['Code']

    url = f'https://v6.exchangerate-api.com/v6/{api_key}/pair/{base_currency}/{curr}'
    response = requests.get(url)

    if response.status_code == 200: 
        my_data = response.json()
        rate_extract[curr] = my_data['conversion_rate']
        rate_extract['Rate_ID'] = my_data['time_last_update_unix']
        rate_extract['Rate_Date'] = my_data['time_last_update_utc']
        rate_extract['Base_Currency'] = my_data['base_code']
        
    else:
        print(f"Error fetching exchange rate from {base_currency} to {curr} - Status code: {response.status_code}")
        rate_extract['Comment'] = f'Rate extract result was: {my_data['result']}'
        break # if any api call fails, then the process should stop
rate_extract['Comment'] = f'Rate extract result was: {my_data['result']}'

In [85]:
rate_extracted = pd.DataFrame([rate_extract])
rate_extracted.head()

,Rate_ID,Rate_Date,Base_Currency,Comment,USD,GBP,EUR,CAD,GHS,QAR
0,1727568004,"Sun, 29 Sep 2024 00:00:04 +0000",NGN,Rate extract result was: success,0.00061,0.000453,0.000542,0.000817,0.009577,0.00222


##### Loading the data into PostgreSQL DB

In [103]:
# Credentials
hostname = os.getenv('hostname')
username = os.getenv('username')
password = os.getenv('password')
port = os.getenv('port')
db_name = os.getenv('db_name')

# Create the DB engine
db_url = f'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{db_name}'
engine = create_engine(db_url)

In [105]:
# Defining my datatypes
my_col_type = {
    'Rate_ID': Numeric,
    'Rate_Date': TIMESTAMP(timezone=True),
    'Base_Currency': VARCHAR(3),
    'USD': Float,
    'GBP': Float,
    'EUR': Float,
    'CAD': Float,
    'GHS': Float,
    'QAR': Float,
    'Comment': VARCHAR(50)
}

In [109]:
rate_extracted.to_sql('rate_extracted', engine, index=False, if_exists='append', dtype=my_col_type)
print("Exchange rate data successfully loaded into database!")

Exchange rate data successfully loaded into database!
